In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotion-detection-from-text/tweet_emotions.csv


In [18]:
data=pd.read_csv('/kaggle/input/emotion-detection-from-text/tweet_emotions.csv')
print(data.columns)
data.head()

Index(['tweet_id', 'sentiment', 'content'], dtype='object')


,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [19]:
data['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [20]:
new_df = pd.DataFrame(columns=['content','sentiment'])

# Iterating over the DataFrame and copying rows that match the specific item
for index, row in data.iterrows():
    if row['sentiment'] in ['sadness', 'neutral', 'love', 'happiness', 'anger']:
        new_row = {'content': row['content'], 'sentiment': row['sentiment']}
        new_df.loc[len(new_df.index)] = [row['content'], row['sentiment']] 


In [21]:
len(new_df)

22964

In [22]:
for i in ['sadness', 'neutral', 'love', 'happiness', 'anger']:
    new_df[i] = new_df['sentiment'].apply(lambda x: 1 if x == i else 0)
new_df.head()

,content,sentiment,sadness,neutral,love,happiness,anger
0,Layin n bed with a headache ughhhh...waitin o...,sadness,1,0,0,0,0
1,Funeral ceremony...gloomy friday...,sadness,1,0,0,0,0
2,@dannycastillo We want to trade with someone w...,neutral,0,1,0,0,0
3,"I should be sleep, but im not! thinking about ...",sadness,1,0,0,0,0
4,@charviray Charlene my love. I miss you,sadness,1,0,0,0,0


In [ ]:
X = new_df['content']
nlp = spacy.load("en_core_web_sm")

X_processed = []
for text in X:
    doc = nlp(text)
    processed_text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    X_processed.append(processed_text)

In [31]:
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR

# Load the spaCy model

sadness = new_df['sadness']
neutral = new_df['neutral']
love = new_df['love']
happiness = new_df['happiness']
anger = new_df['anger']

# Split the data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_processed, sadness, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_processed, neutral, test_size=0.2, random_state=42)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_processed, love, test_size=0.2, random_state=42)
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_processed, happiness, test_size=0.2, random_state=42)
X_train5, X_test5, y_train5, y_test5 = train_test_split(X_processed, anger, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train1)
X_test_vectorized = vectorizer.transform(X_test1)

# Train the model
sadness_m = SVR(kernel='linear')
sadness_m.fit(X_train_vectorized, y_train1)

neutral_m = SVR(kernel='linear')
neutral_m.fit(X_train_vectorized, y_train2)

love_m = SVR(kernel='linear')
love_m.fit(X_train_vectorized, y_train3)

happiness_m = SVR(kernel='linear')
happiness_m.fit(X_train_vectorized, y_train4)

anger_m = SVR(kernel='linear')
anger_m.fit(X_train_vectorized, y_train5)

# Test the model
input_tweet = "I had a really good day, it was nice and I had a coffee"
input_processed = ' '.join([token.lemma_ for token in nlp(input_tweet) if not token.is_stop])
input_vectorized = vectorizer.transform([input_processed])
happiness_score = happiness_m.predict(input_vectorized)[0]

# Happiness score is a continuous value between 0 and 1
print(f"The happiness score for the input tweet is: {happiness_score}")


The happiness score for the input tweet is: 0.5407923354518853


In [1]:
input_tweet = "I had a nice day, I dont' know what to do now, I met a cute girl"
input_processed = ' '.join([token.lemma_ for token in nlp(input_tweet) if not token.is_stop])
input_vectorized = vectorizer.transform([input_processed])
print(anger_m.predict(input_vectorized)[0])
print(sadness_m.predict(input_vectorized)[0])
print(neutral_m.predict(input_vectorized)[0])
print(happiness_m.predict(input_vectorized)[0])
print(love_m.predict(input_vectorized)[0])


NameError: name 'nlp' is not defined

In [36]:
import pickle

model = "happiness.pkl"  

with open(model, 'wb') as file:  
    pickle.dump(happiness_m, file)

model = "saddness.pkl"  

with open(model, 'wb') as file:  
    pickle.dump(sadness_m, file)

model = "anger.pkl"  

with open(model, 'wb') as file:  
    pickle.dump(anger_m, file)

model = "love.pkl"  

with open(model, 'wb') as file:  
    pickle.dump(love_m, file)

model = "nuetral.pkl"  

with open(model, 'wb') as file:  
    pickle.dump(neutral_m, file)


In [38]:
with open('vectorizer.pkl', 'wb') as fin:
    pickle.dump(vectorizer, fin)